In [2]:
import pandas as pd
df=pd.read_csv('data/gemstone.csv')

In [3]:
df

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...,...
193568,193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [5]:
df.drop('id',axis=1,inplace=True)

In [6]:
df.head()

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [8]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'x', 'y', 'z',
       'price'],
      dtype='object')

In [9]:
#independent variables
X=df.drop('price',axis=1)

In [10]:
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [14]:
#dependentvariables
y=df[['price']]
y.head()

,price
0,13619
1,13387
2,2772
3,666
4,14453


In [15]:
# Define which columns should be ordinal-encoded and which should be scaled
numericalcols=X.select_dtypes(exclude='object').columns
categoricalcols=X.select_dtypes(include='object').columns

In [16]:
numericalcols

Index(['carat', 'depth', 'table', 'x', 'y', 'z'], dtype='object')

In [17]:
categoricalcols

Index(['cut', 'color', 'clarity'], dtype='object')

In [20]:
# Define the custom ranking for each ordinal variable
for i in categoricalcols:
    print(i,'->',df[i].unique())

cut -> ['Premium' 'Very Good' 'Ideal' 'Good' 'Fair']
color -> ['F' 'J' 'G' 'E' 'D' 'H' 'I']
clarity -> ['VS2' 'SI2' 'VS1' 'SI1' 'IF' 'VVS2' 'VVS1' 'I1']


In [71]:
cutmap=['Fair','Good','Very Good','Premium','Ideal']
colormap=sorted(list(df['color'].unique()))
colormap.reverse()
claritymap=['I1','SI2','SI1','VS2','VS1','VVS2','VVS1','IF']

In [72]:
colormap

['J', 'I', 'H', 'G', 'F', 'E', 'D']

In [73]:
from sklearn.impute import SimpleImputer #handling missing values
from sklearn.preprocessing import StandardScaler #standardization-feature scaling
from sklearn.preprocessing import OrdinalEncoder # ordinal encoding
from sklearn.pipeline import Pipeline # to create pipeline
from sklearn.compose import ColumnTransformer # to combine numerical and categorical pipeline


In [74]:
numericalpipeline=Pipeline(
    [
        ('imputer',SimpleImputer(strategy='median')), #here median bcz to handle outliers median is best in numerical data
        ('scaler',StandardScaler())
    ]
)
categoricalpipeline=Pipeline(
    [
        ('imputer',SimpleImputer(strategy='most_frequent')),
        ('ordinalencoder',OrdinalEncoder(categories=[cutmap,colormap,claritymap])),
        ('scaler',StandardScaler())
        
    ]
)

preprocessor=ColumnTransformer(
    [
        ('numericalpipeline',numericalpipeline,numericalcols),
        ('categoricalpipeline',categoricalpipeline,categoricalcols)
    ]
)

In [75]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.30,random_state=42)

In [76]:
X_train=pd.DataFrame(preprocessor.fit_transform(X_train),columns=preprocessor.get_feature_names_out())
X_test=pd.DataFrame(preprocessor.transform(X_test),columns=preprocessor.get_feature_names_out())

In [77]:
X_train.head()

,numericalpipeline__carat,numericalpipeline__depth,numericalpipeline__table,numericalpipeline__x,numericalpipeline__y,numericalpipeline__z,categoricalpipeline__cut,categoricalpipeline__color,categoricalpipeline__clarity
0,-0.823144,-1.129988,-0.641897,-0.780451,-0.835103,-0.876024,0.874100,0.936747,1.350746
1,0.945023,-1.777823,0.921902,1.073226,1.166389,0.946633,-1.137644,-0.910853,0.684455
2,1.958484,0.165682,0.400636,1.703116,1.755063,1.742237,-0.131772,-0.910853,0.018164
3,-0.995648,-0.574701,-0.641897,-1.122391,-1.161138,-1.165334,0.874100,0.320880,2.017037
4,-0.995648,0.258230,0.400636,-1.176382,-1.152082,-1.136403,-1.137644,-1.526720,-0.648127


In [78]:
X_test.head()

,numericalpipeline__carat,numericalpipeline__depth,numericalpipeline__table,numericalpipeline__x,numericalpipeline__y,numericalpipeline__z,categoricalpipeline__cut,categoricalpipeline__color,categoricalpipeline__clarity
0,-0.629077,0.258230,-0.120630,-0.600482,-0.581521,-0.572248,0.874100,1.552614,-0.648127
1,2.605374,-2.148014,-0.120630,2.126042,2.198832,1.959219,-1.137644,-0.294987,-1.314417
2,-1.125026,-1.222536,0.921902,-1.374347,-1.414721,-1.469110,-0.131772,0.936747,2.017037
3,-1.017211,-0.574701,0.921902,-1.158385,-1.161138,-1.194265,-0.131772,-1.526720,2.017037
4,0.858771,0.628421,-0.641897,0.947248,0.985258,1.004495,0.874100,-0.910853,-0.648127


In [79]:
#model training
from sklearn.linear_model import LinearRegression,Lasso,Ridge,ElasticNet
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [80]:
lr=LinearRegression()
lr.fit(X_train,y_train)
y_pred=lr.predict(X_test)
mean_squared_error(y_test,y_pred)

1029473.3531156889

In [81]:
y_pred

array([[ 1616.03275998],
       [15104.13631181],
       [ 1727.49228115],
       ...,
       [ 1878.27425152],
       [ 6295.06951547],
       [ 5976.94207688]])

In [82]:
lr.coef_

array([[ 6432.97591819,  -132.34206204,   -70.48787525, -1701.38593925,
         -494.17005097,   -76.32351645,    68.80035873,   464.67990411,
          652.10059539]])

In [83]:
lr.intercept_

array([3976.8787389])

In [86]:
import numpy as np
def modelevaluation(true,predicted):
    mse=mean_squared_error(true,predicted)
    mae=mean_absolute_error(true,predicted)
    rmse=np.sqrt(mse)
    r2=r2_score(true,predicted)
    adjustedr2=1-((1-r2)*(len(X)-1))/(len(X)-X.shape[1]-1)
    return mse,mae,rmse,r2,adjustedr2

In [87]:
#train multiple models
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'ElasticNet':ElasticNet()
}
adjustedr2list=dict()
for i in range(len(models)):
    model=list(models.values())[i]
    model.fit(X_train,y_train)
    y_pred=model.predict(X_test)
    mse,mae,rmse,r2,adjustedr2=modelevaluation(y_test,y_pred)
    print('Model Training Performance')
    print(list(models.keys())[i])
    print("MSE:",mse)
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2*100)
    print("Adjusted R2 score",adjustedr2*100)
    adjustedr2list[list(models.keys())[i]]=adjustedr2
    print('\n')

Model Training Performance
LinearRegression
MSE: 1029473.3531156889
RMSE: 1014.6296630375483
MAE: 675.0758270067445
R2 score 93.62906819996046
Adjusted R2 score 93.62877197399682


Model Training Performance
Lasso
MSE: 1029533.150650549
RMSE: 1014.6591302750638
MAE: 676.2421173665505
R2 score 93.62869814082755
Adjusted R2 score 93.62840189765745


Model Training Performance
Ridge
MSE: 1029482.8101268972
RMSE: 1014.634323353442
MAE: 675.1077629781348
R2 score 93.6290096749163
Adjusted R2 score 93.62871344623146


Model Training Performance
ElasticNet
MSE: 2351174.871397875
RMSE: 1533.3541245902313
MAE: 1060.9432977143008
R2 score 85.44967219374031
Adjusted R2 score 85.44899565457604




In [88]:
adjustedr2list

{'LinearRegression': 0.9362877197399682,
 'Lasso': 0.9362840189765745,
 'Ridge': 0.9362871344623146,
 'ElasticNet': 0.8544899565457603}